<img src="https://github.com/koulali/ceg1705/raw/main/ceg1705_logo_notebook.png" width="1000">


# Practical 3 : Coordinate transformations

The purpose of this  practical is to experiment with transformations between **different datums**, using **cartesian**, **ellipsoidal** and map projection coordinates. You will need to refer to your notes and handouts for some of the formulae. You are welcome to collaborate with a practical partner, but should do individual computations separately, and submit **individual notebooks**. Parts 1, 2 and 3 must be done in that order (Parts 2 and 3 are much quicker than Part 1).

👇 Please, before you start run the following cell to load all the needed functions for this practical.


In [ ]:
!pip install osgb
!pip install pycegm

In [ ]:
# here we load all the modules and functions you need
import numpy as np
import json
from osgb import ll_to_grid,grid_to_ll
from numpy import radians,degrees
from pycegm import units,coords,display

## Part 1.  3-D Helmert (7-parameter) transformations

In this part, you will convert **ellipsoidal coordinates** on the **WGS-84 datum** (as read from a GPS receiver) into **cartesian coordinates**, then transform these into coordinates on the **OSGB36 datum** (as used for most UK maps). Finally you will express your **OSGB36** coordinates in ellipsoidal form, and project them onto the UK National Grid. Recall this flowchart from **lecture 14**

<img src="https://github.com/koulali/ceg1705/blob/main/prac3_fig1.png?raw=true" width="1100">

Everyone has the same WGS-84 coordinates for Points A and B, but C is defined using your birthdate.


| Point A     |             |
| --------------- | --------------- |
| Latitude $\phi$   | 50$^{\circ}$ 30' 42".0000   N  |
| Longitude $\lambda$  | $3^{\circ}$ 59' 59".00000 W	        |
| Height    $h$  | 0.0 m        |

| Point B     |             |
| --------------- | --------------- |
| Latitude $\phi$   | 58$^{\circ}$ 13' 43".49422	N   |
| Longitude $\lambda$  | $4^{\circ}$ 59' 00".03858 W	        |
| Height    $h$  | 0.0 m     

| Point C     |             |
| --------------- | --------------- |
| Latitude $\phi$   | 50$^{\circ}$ 28' MM".00000  N   |
| Longitude $\lambda$  | $3^{\circ}$ 55' DD".00000 W	        |
| Height    $h$  | 0.0 m     



> **where MM and DD are the month and day on which you were born (eg 10 and 03 for October 3rd).**


### (a) Ellipsoidal to Cartesian conversion

Cartesian WGS-84 coordinates of the point.  Note these down on your answer sheet.


> Make sure you use the WGS-84 ellipsoid parameters:  a = 6378137.000 m, b = 6356752.314 m


N.B 

- Cells with "👇 ✍🏻" indicate that you have to input your answer in the following cell.
- Cells with "✏️ Your answer here" where you must write your comments.

#### **Step 1 :Input coordinates ("start here" in the chart)**

We define the Earth radius in kilometers 👇 ✍🏻 

In [ ]:
R = ...

We set the ellipsoid parameters 👇 ✍🏻 

In [ ]:
a = ...
b = ...

Let's define the coordinates of point **A** first.

In [ ]:
# latitude of point A
phi_A = {"deg":...,
         "min":...,
         "sec":...,  
         "dir":"N"}

# longitude of point A
lambda_A = {"deg":...,
            "min":...,
            "sec":...,  
            "dir":"W"}

# height of point A
h_A = 0.0

Here, I prepared for you a function which converts lat/lon cooridnates to cartesian XYZ. For details refer to **practical 2b**. All you have to do is run the following 2 cells. The first does the conversion and the second displays the results

In [ ]:
X_A,Y_A,Z_A = coords.llh_to_xyz(phi_A,lambda_A,h_A,a,b)

In [ ]:
display.print_xyz(X_A,Y_A,Z_A,"Cartesian WGS-84 coordinates of point A")

#### Step 2:

Now your turn, repeat step 1 but for the coordinates B and C.

**B point** 👇 ✍🏻 

In [ ]:
phi_B = {"deg":...,
         "min":...,
         "sec":...,  
         "dir":"N"
        }

lambda_B = {"deg":...,
            "min":...,
            "sec":...,  
            "dir":"W"}

h_B = 0.0

In [ ]:
X_B,Y_B,Z_B = coords.llh_to_xyz(phi_B,lambda_B,h_B,a,b)

In [ ]:
display.print_xyz(X_B,Y_B,Z_B,"Cartesian WGS-84 coordinates of point B")

**C point** 👇 ✍🏻 

In [ ]:
phi_C = {"deg":...,
         "min":...,
         "sec":...,  
         "dir":"N"
        }

lambda_C = {"deg":3,
            "min":55,
            "sec":20,  
            "dir":"W"}

h_C = 0.0

In [ ]:
X_C,Y_C,Z_C = coords.llh_to_xyz(phi_C,lambda_C,h_C,a,b)

In [ ]:
display.print_xyz(X_C,Y_C,Z_C,"Cartesian WGS-84 coordinates of point C")

### (b) Helmert transformation

In order to perform the Helmert transformation, you need  first to find the parameters of the Helmert transformation linking WGS-84 and OSGB36.  These are available in Section 6.6 of the OS document [A guide to coordinate systems in Great Britain](https://www.ordnancesurvey.co.uk/documents/resources/guide-coordinate-systems-great-britain.pdf).

Enter the values of **Tx, Ty, Tz, Rx, Ry, Rz and s** in the cell below. 


> Rx, Ry, Rz were called $\theta_x$, $\theta_y$, $\theta_z$ in lecture 14.


Helmert parameter 👇 ✍🏻 

In [ ]:
# 3 translations, 3 rotations and 1 scale
Tx = ...
Ty = ...
Tz = ...
Rotx = ...
Roty = ...
Rotz = ...
scale  = ...

Rotations `Rotx`, `Roty`, and `Rotz` variables are seconds, so you need to convert to decimal degrees. The new rotation variables will be named `Rx,Ry and Ry` (next cell)

👇 ✍🏻

In [ ]:
Rx = ...
Ry = ...
Rz = ...

The OS scale is in parts per million, so you need to multiply its value by $10^{-6}$ (in python `1e-6`) when using it in the transformation matrix. We create a new variable `s` to be the converted scale.


👇 ✍🏻

In [ ]:
s = ...

Now we need to construct the matrix R (cf. Lecture notes). Please refer to Tutorial 2 in canvas for an introduction to matrix and array computations in python.

**to define rotation matrix $R$ and the position array $p=(X,Y,Z)$**. Replace `...` with each element of the matrix R as in your lecture notes.

👇 ✍🏻

In [ ]:
# rotation matrix
R = np.array([[...,...,...],
              [...,...,...],
              [...,...,...]])

Now Let's define the cartesian coordinates (X,Y,Z) arrays for points A,B and C (in mumpy format). Each array will contain 2 inputs as for X, Y and Z

👇 ✍🏻

In [ ]:
# A point
A = np.array([X_A,Y_A,Z_A])

# B point
B = np.array([X_B,Y_B,Z_B])

# C point
C = np.array([X_C,Y_C,Z_C])

The final Cartesian OSGB36 coordinates are given by the results of your matrix multiplication $R\cdot p$, plus the appropriate translation parameters $[T_x,T_y,T_z]$.

An example of this operation for point D with coordinates array:

```
X,Y,Z = R.dot(D) + np.array([Tx,Ty,Tz])
```


👇 ✍🏻

In [ ]:
# A point
X_A_osgb,Y_A_osgb,Z_A_osgb = ...

# B point
X_B_osgb,Y_B_osgb,Z_B_osgb = ...

# C point
X_C_osgb,Y_C_osgb,Z_C_osgb = ...

In [ ]:
display.print_xyz(X_A_osgb,Y_A_osgb,Z_A_osgb,"Cartesian OSGB36 coordinates of point A")
display.print_xyz(X_B_osgb,Y_B_osgb,Z_B_osgb,"Cartesian OSGB36 coordinates of point B")
display.print_xyz(X_C_osgb,Y_C_osgb,Z_C_osgb,"Cartesian OSGB36 coordinates of point C")

### (c) Cartesian to Ellipsoidal conversion

Now we need to convert the OSGB36 Cartesian coordinates to obtain OSGB36 ellipsoidal coordinates of the point, using the Airy ellipsoid. 

**The Airy ellipsoid parameters:**

a = 6 377 563.396 m

b = 6 356 256.910 m

This is similar to what you did in practical 1. I prepared for you a function which does this as follows:

```
phi,lambda,h = coords.xyz_to_llh(X,Y,Z,a,b)
```

where X,Y,Z are the cartesian input coordinates and the phi,lambda,h are the output ellipsoidal coordinates.


We first define the Airy ellipsoidal parameters a and b

👇 ✍🏻

In [ ]:
a = ...
b = ...

Now do the conversion using the function `xyz_to_llh`. An example for using this function:

```
phi_D_osgb,lambda_D_osgb,h_D_osgb = coords.xyz_to_llh(X_D_osgb,Y_D_osgb,Z_D_osgb,a,b)
```

where the function takes as input the coordinates X,Y,Z and the ellipsoidal parameters a and b

👇 ✍🏻

In [ ]:
phi_A_osgb,lambda_A_osgb,h_A_osgb = ...
phi_B_osgb,lambda_B_osgb,h_B_osgb = ...
phi_C_osgb,lambda_C_osgb,h_C_osgb = ...

In [ ]:
#--- A
display.print_xyz(json.dumps(phi_A_osgb,indent=4, default=str),
             json.dumps(lambda_A_osgb,indent=4, default=str),
             json.dumps(h_A_osgb,indent=4,default=str),
             "Ellipsoidal OSGB36 coordinates of point A",type="ll")


#--- B
display.print_xyz(json.dumps(phi_B_osgb,indent=4, default=str),
             json.dumps(lambda_B_osgb,indent=4, default=str),
             json.dumps(h_B_osgb,indent=4,default=str),
             "Ellipsoidal OSGB36 coordinates of point B",type="ll")


#--- C
display.print_xyz(json.dumps(phi_C_osgb,indent=4, default=str),
             json.dumps(lambda_C_osgb,indent=4, default=str),
             json.dumps(h_C_osgb,indent=4,default=str),
             "Ellipsoidal OSGB36 coordinates of point C",type="ll")

### (d) Map projection (OSGB36) coordinates

Here, we project OSGB36 Airy ellipsoid coordinates onto the National Grid (it can also be modified to use other Transverse Mercator projections such as UTM). We need to obtain OSGB36 National Grid coordinates (**eastings and northings**). For this projection we will use the function `ll_to_os`.

Again, to call the `ll_to_os` function we simply do:

```
    easting,northing = coords.ll_to_grid(lat,lon)
```

Let's first convert OSGB36 ellipsoidal coordinates to decimal degrees.The variables in decimal degrees will be named using this schema: `phi_{point name}_osgb_dec` 

Run the cell below

In [ ]:
# A point
phi_A_osgb_dec    = units.dms_to_decimal(phi_A_osgb)
lambda_A_osgb_dec = units.dms_to_decimal(lambda_A_osgb)

# B point
phi_B_osgb_dec    = units.dms_to_decimal(phi_B_osgb)
lambda_B_osgb_dec = units.dms_to_decimal(lambda_B_osgb)

# C point
phi_C_osgb_dec    = units.dms_to_decimal(phi_C_osgb)
lambda_C_osgb_dec = units.dms_to_decimal(lambda_C_osgb)

You can now convert the OSGB36 latitude,longitude [decimal degrees] to easting and northing

👇 ✍🏻

In [ ]:
easting_A, northing_A = ...
easting_B, northing_B = ...
easting_C, northing_C = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(easting_A,northing_A,h_A_osgb,"Local E,N OSGB36 coordinates of point A",type="en")
display.print_xyz(easting_B,northing_B,h_B_osgb,"Local E,N OSGB36 coordinates of point B",type="en")
display.print_xyz(easting_C,northing_C,h_C_osgb,"Local E,N OSGB36 coordinates of point C",type="en")

## **We reached the end of the chart**


You have now the eastings and northings for Points B and C, compute the **baseline vector from Point A to them** (the difference in eastings and northings between Points B/C and Point A).

<img src="https://github.com/koulali/ceg1705/blob/main/prac3_fig2.png?raw=true" width="600">

Baseline vector $\Delta AB$

👇 ✍🏻

In [ ]:
Delta_easting_AB  = ...
Delta_northing_AB = ...
Delta_h_AB = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(Delta_easting_AB,Delta_northing_AB,Delta_h_AB,"Relative OSGB36 coords baseline A→B:",type="ba")

Baseline vector $\Delta AC$

👇 ✍🏻

In [ ]:
Delta_easting_AC  = ...
Delta_northing_AC = ...
Delta_h_AC = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(Delta_easting_AC,Delta_northing_AC,Delta_h_AC,"Relative OSGB36 coords baseline A→C:",type="ba")


> What do you notice about the apparent E/W position of A and B as described by their WGS-84 and OSGB36 longitudes, and OSGB36 easting?  Why do these change?


✏️ **Your answer here**

...

...

...

...

---

## Part 2.  Simpler 3-D transformations

In this part, you will estimate the parameters of a simpler transformation that might be of use to navigators near Point A.  Using Points B and C, you will then be able to compare with the results from Part 1 to test the validity of this simpler transformation near to Point A and further away.

### Step 1

By comparing the Cartesian coordinates of Point A in the WGS-84 and OSGB36 datums, deduce the translation parameters (Tx, Ty, Tz) that should be added to Point A’s Cartesian WGS-84 coordinates in order to transform them to OSGB36, if the scale and rotation parameters were set to zero (i.e. not used). We call the new translation parameters here `Tnx,Tny and Tnz`


👇 ✍🏻

In [ ]:
Tnx = ...
Tny = ...
Tnz = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(Tnx,Tny,Tnz,"Translation-only Helmert parameters using point A:",type="ba")

### Step 2
Use these translation parameters (setting scale and rotation to zero) to transform Points B and C’s WGS-84 Cartesian coordinates into OSGB36.


- Calculate Point B cartesian coordinates:

👇 ✍🏻 

In [ ]:
X_B_osgb_tr = ...
Y_B_osgb_tr = ...
Z_B_osgb_tr = ...

- Calculate Point C cartesian coordinates: 

👇 ✍🏻 

In [ ]:
X_C_osgb_tr = ...
Y_C_osgb_tr = ...
Z_C_osgb_tr = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(X_B_osgb_tr,Y_B_osgb_tr,Z_B_osgb_tr,"Cartesian OSGB36 coords point B:",type="xyz")
display.print_xyz(X_C_osgb_tr,Y_C_osgb_tr,Z_C_osgb_tr,"Cartesian OSGB36 coords point C:",type="xyz")

**Thence calculate National Grid OSGB36 coordinates for these points**. To do this, we need first to calculate the ellipsoidal coordinates in OSGB36 for the newly translated points.


Ellipsoidal coordinates for Point B (`X_B_osgb_tr`) first

👇 ✍🏻 

In [ ]:
# First convert to lat,lon 
# Point B
phi_B_osgb_tr,lambda_B_osgb_tr,h_B_osgb_tr = ...

Then, Ellipsoidal coordinates for Point C (`X_C_osgb_tr`)

👇 ✍🏻 

In [ ]:
# First convert to lat,lon 
# Point B
phi_C_osgb_tr,lambda_C_osgb_tr,h_C_osgb_tr = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(phi_B_osgb_tr,lambda_B_osgb_tr,h_B_osgb_tr,"OSGB36 ellipsoidal coords point B:",type="ll")
display.print_xyz(phi_C_osgb_tr,lambda_C_osgb_tr,h_C_osgb_tr,"OSGB36 ellipsoidal coords point C:",type="ll")

In [ ]:
# convert to decimal coordinates

# B point
phi_B_osgb_tr_dec    = units.dms_to_decimal(phi_B_osgb_tr)
lambda_B_osgb_tr_dec = units.dms_to_decimal(lambda_B_osgb_tr)


# C point
phi_C_osgb_tr_dec    = units.dms_to_decimal(phi_C_osgb_tr)
lambda_C_osgb_tr_dec = units.dms_to_decimal(lambda_C_osgb_tr)

Now apply the function `ll_to_grid` to get the northing and the easting


👇 ✍🏻

In [ ]:
# Caculate National grid coordinates
easting_B_tr, northing_B_tr = ...
easting_C_tr, northing_C_tr = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(easting_B_tr, northing_B_tr,h_B_osgb_tr,"Projection OSGB36 coords point B:",type="ll")
display.print_xyz(easting_B_tr, northing_B_tr,h_C_osgb_tr,"Projection OSGB36 coords point C:",type="ll")

### Step 3

By how much do these (presumably somewhat less accurate) grid coordinates for Points B and C differ from the answers in Part 1?

Difference in coordinates $\delta B$ (easting,northing and height). We call this variables `diff_{E,N,h}_B`

👇 ✍🏻

In [ ]:
diff_E_B = ...
diff_N_B = ...
diff_h_B = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(diff_E_B, diff_N_B,diff_h_B,"Differences in OSGB36 coords, with respect to Part 1, Point B",type="dif")

Difference in coordinates $\delta C$

👇 ✍🏻

In [ ]:
diff_E_C = ...
diff_N_C = ...
diff_h_C = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(diff_E_C, diff_N_C,diff_h_C,"Differences in OSGB36 coords, with respect to Part 1, Point C",type="dif")


> **How do these differences compare with the precision of various GPS techniques?**


✏️ **Your answer here**


...

...

...

...

---

## Part 3.  The OSTN15 transformation

The OS provide a precise transformation (in fact definitive, and good to ~20 cm in plan and ~2 cm in height compared with old mapping) which takes into account the distortions in the making of OSGB36.  One way to use this is via the “Coordinate transformation tool” on the OS GPS website (http://www.ordnancesurvey.co.uk/gps/transformation/). Use this to get exact National Grid coordinates for Points A, B and C (in this module don’t worry about the small differences between ETRS89 and WGS-84). 

Ipnut the east/north coordinates obtained from the online tool below:

👇 ✍🏻

In [ ]:
easting_A_tool  = ...
northing_A_tool = ...
h_A_tool        = ...

easting_B_tool  = ...
northing_B_tool = ...
h_B_tool        = ...

easting_C_tool  = ...
northing_C_tool = ...
h_C_tool        = ...

### Step 1: Calculate the differences in OSGB36 coords (“absolute” i.e. Part 3 minus Part 1):


Differences $\delta A$

👇 ✍🏻

In [ ]:
diff_E_abs_A = ...
diff_N_abs_A = ...
diff_h_abs_A = ...

Differences $\delta B$

👇 ✍🏻

In [ ]:
diff_E_abs_B = ...
diff_N_abs_B = ...
diff_h_abs_B = ...

Differences $\delta C$

👇 ✍🏻

In [ ]:
diff_E_abs_C = ...
diff_N_abs_C = ...
diff_h_abs_C = ...

In [ ]:
# DO NOT MODIFY THIS CELL
display.print_xyz(diff_E_abs_A,diff_N_abs_A,diff_h_abs_A,"Differences in OSGB36 coords (“absolute”), A:",type="dif")
display.print_xyz(diff_E_abs_B,diff_N_abs_B,diff_h_abs_B,"Differences in OSGB36 coords (“absolute”), B:",type="dif")
display.print_xyz(diff_E_abs_C,diff_N_abs_C,diff_h_abs_C,"Differences in OSGB36 coords (“absolute”), C:",type="dif")

### Step 2: Calculate the relative coordinates (the baselines joining Point A to Points B and C)  

“absolute” i.e. Part 3 minus Part 1:

First, the OSGB36 baselines (from the online tool) (baselines A→B = B-A and A→C = C-A)

Baseline vector $\Delta AB$

👇 ✍🏻


In [ ]:
Delta_easting_AB_rel = ...
Delta_northing_AB_rel = ...
Delta_h_AB_rel = ...

Baseline vector $\Delta AC$

👇 ✍🏻

In [ ]:
Delta_easting_AC_rel = ...
Delta_northing_AC_rel = ...
Delta_h_AC_rel = ...

In [ ]:
display.print_xyz(Delta_easting_AC_rel,Delta_northing_AC_rel,Delta_h_AC_rel,"Relative OSGB36 coords baseline A→C:",type="ba")

Now the baseline differences $d\Delta E$ and $d\Delta N$ in baselines (Differences between Parts 1 and 3 )

$d\Delta AB (E,N)$

👇 ✍🏻

In [ ]:
diff_easting_AB = ...
diff_northing_AB = ...
diff_h_AB = ...

In [ ]:
display.print_xyz(diff_easting_AB,diff_northing_AB,diff_h_AB,"Relative OSGB36 coords baseline A→B:",type="dif")

$d\Delta AC (E,N)$

👇 ✍🏻

In [ ]:
diff_easting_AC = ...
diff_northing_AC = ...
diff_h_AC = ...

In [ ]:
display.print_xyz(diff_easting_AC,diff_northing_AC,diff_h_AC,"Relative OSGB36 coords baseline A→C:",type="dif")


>Comment on the size of the differences in absolute coordinates (actual E, N, h of Points A, B and C separately) and relative coordinates (the baselines joining Point A to >Points B and C) compared with your answers from Parts 1 and 2.  Compare your answers with the precision of various GNSS techniques that you have learned about in this module.


✏️ **Your answer here**

...

...

...

...



🎉🎉🎉🎉 The END


This practical forms part of the assessed coursework and is worth up to 5\% of the total CEG1705/CEG2719 module mark. 💎💎💎💎💎


Share your work via canvas Practical 3 submission page.